In [1]:
pretty = True
highres = True

%matplotlib inline
if highres:
    %config InlineBackend.figure_format = 'retina'
else:
    %config InlineBackend.figure_format = 'png'

#rcParams["figure.dpi"]=300

import sys
from pathlib import Path

pypsapath = "C:/dev/py/PyPSA/"

if sys.path[0] != pypsapath:
    sys.path.insert(0,pypsapath)

if Path("../..") not in [Path(p) for p in sys.path]:
    sys.path.insert(0,"../..")

%load_ext autoreload
%autoreload 2

In [2]:
import src.globals
from src.scigridnetwork import SciGRID_network
from src.armafitloader import ARMAfit_loader
import src.plothelper

Git root path found at: C:\dev\grid-analysis
Using data path:        C:\dev\grid-analysis\data


In [3]:
import pypsa
import numpy as np
import pandas as pd
import os
import itertools
import scipy.stats

import matplotlib.pyplot as plt
import matplotlib.ticker
import matplotlib.dates
import matplotlib.font_manager
from matplotlib import rcParams

rcParams["font.family"] = "sans-serif"


from IPython.display import Markdown, display
printm = lambda s: display(Markdown(s))

In [4]:
sgn = SciGRID_network()
fav_sgn = sgn

Importing PyPSA from older version of PyPSA than current version 0.13.2.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.



0.13.2 ['C:/dev/py/PyPSA\\pypsa']


In [5]:
month_index = 0
month_name = ARMAfit_loader.monthnames[month_index]

solar_diff_cov = np.load(src.globals.data_path / "processed" / "covariance" / month_name / "solar_diffdaylightcov.npy")
wind_diff_cov = np.load(src.globals.data_path / "processed" / "covariance" / month_name / "wind_diffcov.npy")
solar_diff_cov_norm = np.load(src.globals.data_path / "processed" / "covariance" / month_name / "solar_diffdaylightcovnorm.npy")
wind_diff_cov_norm = np.load(src.globals.data_path / "processed" / "covariance" / month_name / "wind_diffcovnorm.npy")

In [6]:
bus_diff_cov_day = wind_diff_cov + solar_diff_cov
bus_diff_cov_night = wind_diff_cov
bus_diff_cov_day_norm = wind_diff_cov_norm + solar_diff_cov_norm
bus_diff_cov_night_norm = wind_diff_cov_norm

In [7]:
line_diff_cov_day = sgn.F @ bus_diff_cov_day @ sgn.F.T
line_diff_cov_night = sgn.F @ bus_diff_cov_night @ sgn.F.T
line_diff_cov_day_norm = sgn.F @ bus_diff_cov_day_norm @ sgn.F.T
line_diff_cov_night_norm = sgn.F @ bus_diff_cov_night_norm @ sgn.F.T

In [8]:
sgn.run_lopf_jan1()

Performing linear OPF for one day, 4 snapshots at a time:


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:47<00:00,  7.81s/it]


In [73]:
solar_total = pd.DataFrame(0.0, columns=range(sgn.n), index=sgn.network.generators_t.p.index[:24])
onshore_total = pd.DataFrame(0.0, columns=range(sgn.n), index=sgn.network.generators_t.p.index[:24])
offshore_total = pd.DataFrame(0.0, columns=range(sgn.n), index=sgn.network.generators_t.p.index[:24])

for gen_name, series in sgn.network.generators_t.p.iteritems():
    i = sgn.node_index(sgn.network.generators.bus[gen_name])
    
    source = sgn.network.generators.source[gen_name]
    
    if source == "Solar":
        solar_total[i] += series[:24]
    if source == "Wind Onshore":
        onshore_total[i] += series[:24]
    if source == "Wind Offshore":
        offshore_total[i] += series[:24]

current_use = solar_total.iloc[11,:] + onshore_total.iloc[11,:] + offshore_total.iloc[11,:]
stochastic_capacity = sgn.solar_capacity + sgn.wind_capacity

In [103]:
will_disconnect = np.abs(np.diag(sgn.M)) < 1e-8

In [9]:
ratings = sgn.get_line_ratings(bus_diff_cov_day)

In [142]:
rs = ratings.sort_values("P>1", ascending=False).head(40).copy()



rs["totalchange"] = 0.0
rs["numbusextendedf"] = 0.0
rs["numjoint"] = 0.0
rs["numcascaded"] = 0.0
rs["cascade"] = 0.0

rs["willdisconnectf"] = "❌"
rs["jointwilldisconnectf"] = "❌"
rs["finaldisconnectedf"] = "❌"

time = sgn.network.generators_t.p.index[11]
mup = sgn.injection_total.loc[time]

def lines_disconnect(lines):
    return np.linalg.matrix_rank(sgn.M[lines,:][:,lines], tol=1e-8) != len(lines)

for i, l in enumerate(rs.index):
    mlinjection = sgn.most_likely_power_injection_given_line_failure(l, bus_diff_cov_day)
    diff = mlinjection - mup

    rs.loc[l,"totalchange"] = np.sum(np.abs(diff))# / np.sum(np.abs(mup))
    
    new_renewable_gen = current_use + diff
    
    under_capacity = new_renewable_gen < 0
    over_capacity = new_renewable_gen > stochastic_capacity
    
    average_extension = np.sum(np.abs(list(new_renewable_gen[under_capacity]) + list((new_renewable_gen - stochastic_capacity)[over_capacity])))
    
    rs.loc[l, "numbusextendedf"] = "{:n} ({:.0f} MW)".format(np.sum(under_capacity) + np.sum(over_capacity), average_extension)
    
    casc = list(sgn.simulate_cascade(l, bus_diff_cov_day))
    
    rs.loc[l, "numjoint"] = len(casc[0][0])
    
    rs.loc[l, "numcascaded"] = len(casc[-1][0])
    
    cascadeconnected = []
    cascadedisconnected = []
    
    for z in casc:
        failed = z[0]
        if lines_disconnect(failed):
            cascadedisconnected.append(len(failed))
        else:
            cascadeconnected.append(len(failed))
    
    cascstring = " -> ".join(map(str,cascadeconnected))
    if cascadedisconnected:
        if cascadeconnected:
            cascstring += " -> "
        if len(cascadedisconnected) > 4:
            cascadedisconnected = cascadedisconnected[:2] + ["...", cascadedisconnected[-1]]
        cascstring += "(" + " -> ".join(map(str,cascadedisconnected)) + ")"
    
    
    rs.loc[l, "cascade"] = cascstring
    rs.loc[l, "willdisconnectf"] = "yes" if will_disconnect[l] else "no"
    
    joint = casc[0][0]
    
    rs.loc[l, "jointwilldisconnectf"] = "yes" if lines_disconnect(joint) else "no"
    
    final = casc[-1][0]
    
    rs.loc[l, "finaldisconnectedf"] = "yes" if lines_disconnect(final) else "no"


rs["ff"] = ["{:.2f}".format(np.abs(f)) for f in rs.f]
rs["thresholdf"] = ["{:.1f} GW".format(sgn.line_threshold[l] / 1e3) for l in rs.index]
rs["σf"] = ["{:.2f}".format(np.abs(σ)) for σ in rs.σ]

rs["P>1f"] = ["{:6.4f}%".format(100*x) for x in rs["P>1"]]

rs["totalchangef"] = ["{:.1f} GW".format(np.abs(totalchange)/1e3) for totalchange in rs.totalchange]

rs["numjointf"] = ["{:n}".format(numjoint) for numjoint in rs.numjoint]
rs["numcascadedf"] = ["{:n}".format(numcascaded) for numcascaded in rs.numcascaded]

# with pd.option_context("max_colwidth", 1000):
#     rs[["P>1f","ff","thresholdf","σf","totalchangef", "numbusextendedf", "numjointf", "numcascadedf", "cascade", "willdisconnectf", "jointwilldisconnectf", "finaldisconnectedf"]].to_latex("results.tex")
"🐟"

In [179]:
top10 = list(rs.index[:10])
top10old = [sgn.scigrid_line_indices[i][0] for i in top10]

lengths = sgn.network.lines.length_m.loc[top10old]
thresholds = sgn.network.lines.s_nom.loc[top10old]
susceptances = sgn.i_times_lineadmittance_new[top10]

resistances = sgn.network.lines.r.loc[top10old]
reactances = sgn.network.lines.x.loc[top10old]

print("Lengths:")
print("top10: mean {:.0f} SD {:.0f}".format(np.mean(lengths/1e3), np.std(lengths/1e3)))
print("all: mean {:.0f} SD {:.0f}".format(np.mean(sgn.line_lengths), np.std(sgn.line_lengths)))
print()
print("Thresholds:")
print("top10: mean {:.0f} SD {:.0f}".format(np.mean(thresholds), np.std(thresholds)))
print("all: mean {:.0f} SD {:.0f}".format(np.mean(sgn.line_threshold), np.std(sgn.line_threshold)))
print()
print("Susceptance:")
print("top10: mean {:.0f} SD {:.0f}".format(np.mean(susceptances), np.std(susceptances)))
print("all: mean {:.0f} SD {:.0f}".format(np.mean(sgn.i_times_lineadmittance_new), np.std(sgn.i_times_lineadmittance_new)))
print()
print("Resistance:")
print("top10: mean {:.1f} SD {:.1f}".format(np.mean(resistances), np.std(resistances)))
print("all: mean {:.1f} SD {:.1f}".format(np.mean(sgn.network.lines.r), np.std(sgn.network.lines.r)))
print()
print("Reactance:")
print("top10: mean {:.1f} SD {:.1f}".format(np.mean(reactances), np.std(reactances)))
print("all: mean {:.1f} SD {:.1f}".format(np.mean(sgn.network.lines.x), np.std(sgn.network.lines.x)))

Lengths:
top10: mean 43 SD 30
all: mean 36 SD 35

Thresholds:
top10: mean 554 SD 219
all: mean 1385 SD 1052

Susceptance:
top10: mean 58605 SD 119370
all: mean 613037 SD 2271225

Resistance:
top10: mean 3.9 SD 3.3
all: mean 2.0 SD 2.6

Reactance:
top10: mean 18.9 SD 12.5
all: mean 12.0 SD 13.6


In [174]:
sgn.i_times_lineadmittance_new

(695,)

In [107]:
list(sgn.simulate_cascade(25, bus_diff_cov_day))

[([25, 54], 0     -1.526922e+02
  1      5.927144e+02
  2      5.274443e+02
  3     -8.845935e+02
  4      1.217961e+03
  5      4.001825e+00
  6      5.748101e+01
  7      4.011490e+01
  8     -8.814578e+01
  9     -1.175954e+02
  10     1.341251e+03
  11    -4.494394e+02
  12     4.261744e+02
  13     1.382886e+02
  14    -6.371941e+01
  15     1.274246e+03
  16    -1.787661e+03
  17    -2.725919e+02
  18    -9.565621e+02
  19    -4.276066e+01
  20     4.561208e+00
  21     5.824802e+02
  22    -6.129535e+02
  23     5.861424e+01
  24     1.233165e+03
  25    -5.684342e-13
  26    -9.202592e+02
  27     9.073673e+02
  28     1.674630e+00
  29    -2.423652e+00
             ...     
  665   -4.679504e+01
  666   -2.022102e+00
  667    4.121085e+00
  668    3.133766e+00
  669    1.068351e+02
  670    1.951175e+02
  671    1.756705e+00
  672   -7.680279e+02
  673   -1.742320e+02
  674   -1.421098e+00
  675    1.066025e+02
  676   -3.188009e+01
  677   -7.444378e+01
  678   -9.876788e+01
